In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [2]:
# Load API Credentials
with open("C:/Users/Yosr/Documents/GitHub/Create-and-Save-Your-Yelp-API-Key/.secret/yelp_api.json") as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)



In [3]:
# set our API call parameters 
LOCATION = 'Baltimore'
TERM = 'Seafood'


In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "C:/Users/Yosr/Documents/GitHub/Create-and-Save-Your-Yelp-API-Key/.secret/results_in_progress_Baltimore_Seafood.json"
JSON_FILE



'C:/Users/Yosr/Documents/GitHub/Create-and-Save-Your-Yelp-API-Key/.secret//results_in_progress_Baltimore_Seafood.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] C:/Users/Yosr/Documents/GitHub/Create-and-Save-Your-Yelp-API-Key/.secret//results_in_progress_Baltimore_Seafood.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results



1400

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



70

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [13]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [14]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[i] C:/Users/Yosr/Documents/GitHub/Create-and-Save-Your-Yelp-API-Key/.secret//results_in_progress_Baltimore_Seafood.json not found. Saving empty list to new file.
- 0 previous results found.


70

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/70 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [16]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ieS_5zqxDHcWMCm8BKUYbg,thames-street-oyster-house-baltimore,Thames Street Oyster House,https://s3-media4.fl.yelpcdn.com/bphoto/SLti9z...,False,https://www.yelp.com/biz/thames-street-oyster-...,3138,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.28214, 'longitude': -76.59162}",[delivery],$$$,"{'address1': '1728 Thames St', 'address2': '',...",+14434497726,(443) 449-7726,3377.766396
1,1C8QSqc-erkkyAUywh501A,angies-seafood-baltimore,Angie's Seafood,https://s3-media2.fl.yelpcdn.com/bphoto/cgqaFi...,False,https://www.yelp.com/biz/angies-seafood-baltim...,1024,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.4,"{'latitude': 39.289111, 'longitude': -76.592704}","[pickup, delivery]",$$,"{'address1': '1727 E Pratt St', 'address2': No...",+14103420917,(410) 342-0917,2641.993103
2,BhnCaqbJtWrC36eVxlewSw,l-p-steamers-baltimore,L P Steamers,https://s3-media3.fl.yelpcdn.com/bphoto/Ctttm9...,False,https://www.yelp.com/biz/l-p-steamers-baltimor...,1900,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.2,"{'latitude': 39.27093, 'longitude': -76.59861}",[pickup],$$,"{'address1': '1100 E Fort Ave', 'address2': ''...",+14105769294,(410) 576-9294,4363.353080
3,D9A33FM394q99o4QtK5YwA,faidleys-seafood-baltimore-3,Faidley's Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/Rw16OE...,False,https://www.yelp.com/biz/faidleys-seafood-balt...,1315,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.2,"{'latitude': 39.291696, 'longitude': -76.62224}",[delivery],$$,"{'address1': '203 N Paca St', 'address2': '', ...",+14107274898,(410) 727-4898,2261.836419
4,VLGdENTqjFJuCjg1SbW7_g,nicks-fish-house-baltimore,Nicks Fish House,https://s3-media3.fl.yelpcdn.com/bphoto/_oQF05...,False,https://www.yelp.com/biz/nicks-fish-house-balt...,1189,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.0,"{'latitude': 39.26036, 'longitude': -76.61421}",[pickup],$$,"{'address1': '2600 Insulator Dr', 'address2': ...",+14103474123,(410) 347-4123,5420.410604


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,-EqRC5drgdwZwfvGrg518w,manna-rang-catonsville,Manna Rang,https://s3-media3.fl.yelpcdn.com/bphoto/OHxOwu...,False,https://www.yelp.com/biz/manna-rang-catonsvill...,30,"[{'alias': 'korean', 'title': 'Korean'}]",3.5,"{'latitude': 39.282715, 'longitude': -76.755361}",[delivery],$,"{'address1': '800 N Rolling Rd', 'address2': '...",+14436129029,(443) 612-9029,12933.267765
996,X0FAvERa1RL3XJ4yZwRSyg,nicks-pizza-baltimore,Nick's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/hwzxg5...,False,https://www.yelp.com/biz/nicks-pizza-baltimore...,15,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.1,"{'latitude': 39.2807108978229, 'longitude': -7...","[delivery, pickup]",NaN,"{'address1': '2200 Wilkens', 'address2': '', '...",+14105665333,(410) 566-5333,4785.865272
997,aymXOEor5iwV4K8FDWIuYg,aji-sushi-owings-mills-2,Aji Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/ed0-eU...,False,https://www.yelp.com/biz/aji-sushi-owings-mill...,18,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.3,"{'latitude': 39.4185567, 'longitude': -76.7767...","[delivery, pickup]",NaN,"{'address1': '10347 Reisterstown Rd', 'address...",+14103637720,(410) 363-7720,18857.209657
998,I0NdWFRJBxuxGQgrbwbuVw,china-east-baltimore,China East,https://s3-media3.fl.yelpcdn.com/bphoto/XsqTy7...,False,https://www.yelp.com/biz/china-east-baltimore?...,32,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.7,"{'latitude': 39.28728016654823, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '4712 Eastern Ave', 'address2': '...",+14106336699,(410) 633-6699,5167.014800
999,yllf6v4-mGd2kMx8h7TPRA,szechuan-hot-wok-linthicum-heights,Szechuan Hot Wok,https://s3-media2.fl.yelpcdn.com/bphoto/TalMRp...,False,https://www.yelp.com/biz/szechuan-hot-wok-lint...,35,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",3.4,"{'latitude': 39.2243799, 'longitude': -76.66488}","[delivery, pickup]",$,"{'address1': '831 Nursery Rd', 'address2': '',...",+14106360102,(410) 636-0102,10603.349253


In [18]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()



0

In [20]:
# save the final results to a compressed csv
final_df.to_csv("C:/Users/Yosr/Documents/GitHub/Create-and-Save-Your-Yelp-API-Key/.secret/final_results_Baltimore_Seafood.csv.gz', compression='gzip',index=False)



SyntaxError: unterminated string literal (detected at line 2) (3575634088.py, line 2)